# Protein-protein docking

Trying [pyDockweb](https://life.bsc.es/pid/pydock/) is a very good start. 


# Protein-small molecule docking

## Autodock vina short tutorial 

Check good examples on how to use the software:
* Follow Sari Sabban's [videotutorial](https://www.youtube.com/watch?v=rBEKZQ22nhs) on the use of [autodock](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4868550/) for docking ligands into HIV-1 PROTEASE;
* Also José Manuel Nápoles Duarte [post](https://towardsdatascience.com/identifying-protein-ligand-interactions-with-colab-2cfd26ed469a) on the use of Py3Dmol to visualize results from [autodock vina](http://vina.scripps.edu/manual.html). We use some of his functions here.
* Finally, Jean Didier Maréchal has posted a [nice and complete tutorial](https://gent.uab.cat/jdidier/content/practical-3-protein-ligand-docking-autodock) worth following for more advanced command-line calculations with AutoDock.

We will use the PDB code [1HVC](https://www.rcsb.org/structure/1HVC) as our receptor.

----------------------

Requirements:

* `biopython`. You can get it [through](https://biopython.org/wiki/Packages)
```
conda install -c conda-forge biopython
```
* `openbabel`. Open, collaborative project allowing anyone to search, convert, analyze, or store data from molecular modeling, chemistry, solid-state materials, biochemistry, or related areas. You can get it through
```
conda install -c openbabel openbabel
```
If you are using Mac OS X you may also want to install [`ibabel`](https://www.macinchem.org/ibabel/version4/iBabel4_0.php) for a graphical taste of the program:
```
pip install ibabel
```
* `pymol` (optional). User-sponsored molecular visualization system on an open-source foundation, maintained and distributed by Schrödinger. You will need to get a (free) [student license](https://pymol.org/edu/?q=educational). 
```
conda install -c schrodinger pymol-bundle
```
Alternatively, if you have troubles using pymol (as occurs to me in my Mac OS X) you may have enough with `py3Dmol` as shown in this script. You can install it with `pip`:
```
pip -q install py3Dmol
```
----------------------

Let us start by downloading the receptor PDB file using [Biopython](https://biopython.org/docs/1.75/api/Bio.PDB.PDBList.html#Bio.PDB.PDBList.PDBList.retrieve_pdb_file)



In [ ]:
from Bio.PDB import PDBList

pdbcode = '1HVC'
datadir = 'data'
pdbl = PDBList()
pdbfile = pdbl.retrieve_pdb_file(pdbcode,file_format='pdb',pdir=datadir)
print(pdbfile)

Now we will extract only the lines containing the atoms of the polypeptide chain. Those lines contain the tag ATOM and we will simply use the shell command `grep` to run such extraction. Analogously, we will extract the lines containing the ligand (HETATM tag):

In [ ]:
!grep ATOM {pdbfile} |grep -v HOH > {datadir}/R.pdb
!grep HETATM {pdbfile} |grep -v HOH > {datadir}/L.pdb

Autodock needs the receptor file in a format able to contain the charges for every atom. So, we will convert the recently created `R.pdb`file into a `receptor.pdbqt`file using `openbabel`. We will do the same with the `ligand.pdbqt` file. The files used in this example have zero charges for each ligand atom, which is wrong. I recommend you to use a partial charges calculator like [Atomic Charge Calculator II](https://acc2.ncbr.muni.cz).

In [ ]:
!obabel {datadir}/R.pdb -O {datadir}/temp.pdbqt -o pdbqt -xh
!grep ATOM {datadir}/temp.pdbqt > {datadir}/receptor.pdbqt
!obabel {datadir}/L.pdb -O {datadir}/ligand.pdbqt -o pdbqt -xh

In [ ]:
#checking if everything went right
!tail {datadir}/receptor.pdbqt
!tail {datadir}/ligand.pdbqt

It will be handy to know what is the center of the ligand, as it will be used to create the search box for autodock vina docking

In [ ]:
import numpy
a = numpy.genfromtxt("data/L.pdb", skip_header=1, usecols=[6, 7, 8])
xligand=a.mean(axis=0)[0]
yligand=a.mean(axis=0)[1]
zligand=a.mean(axis=0)[2]
print('The coordinates of the geometric center of the ligand are:',xligand,yligand,zligand)

In [ ]:
import py3Dmol

In [ ]:
def visbox2(objeto,bxi,byi,bzi,bxf,byf,bzf):
    print(bxi,byi,bzi,bxf,byf,bzf)
    objeto.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.5})

def complxvis(objeto,protein_name,ligand_name):
    mol1 = open(protein_name, 'r').read()
    mol2 = open(ligand_name, 'r').read()
    objeto.addModel(mol1,'pdb')
    objeto.setStyle({'cartoon': {'color':'spectrum'}})
    objeto.addModel(mol2,'pdb')
    objeto.setStyle({'model':1},{'stick':{}})

def vismol(bxi=-10,byi=-10,bzi=-10,bxf=5,byf=5,bzf=5):
    mol_view = py3Dmol.view(800, 400,viewergrid=(1,2))  
    visbox2(mol_view,bxi,byi,bzi,bxf,byf,bzf)
    complxvis(mol_view,datadir+'/R.pdb',datadir+'/L.pdb')
    mol_view.setBackgroundColor('0xeeeeee')
    mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
    mol_view.zoomTo()  
    mol_view.show()
  
from ipywidgets import interact,fixed,IntSlider
import ipywidgets
print(xligand,yligand,zligand)
interact(vismol,
         bxi=ipywidgets.IntSlider(min=-100,max=xligand+100, step=1),
         byi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bzi=ipywidgets.IntSlider(min=-100,max=100, step=1),
         bxf=ipywidgets.IntSlider(min=0,max=30, step=1),
         byf=ipywidgets.IntSlider(min=0,max=30, step=1),
         bzf=ipywidgets.IntSlider(min=0,max=30, step=1))

Once we have the PDBQT files, we can run [autodock vina](http://vina.scripps.edu/download.html). The best way to do it command line is taking profit of the configuration file option. From the above calculation, the values to specify the box can be introduced in a configuration file like follows:

```
receptor = data/receptor.pdbqt
ligand = data/ligand.pdbqt

out = data/all.pdbqt

center_x = 5 
center_y = -1
center_z = 15  

size_x = 15
size_y = 16
size_z = 15
```

Let us call the file created with this information `conf.txt`. Thus, running `autodock vina` is as simple as (check where the executable is in your computer, of course, or add it to your `$PATH`: 

In [ ]:
%%bash
cat <<EOF > conf.txt
receptor = data/receptor.pdbqt
ligand = data/ligand.pdbqt

out = data/all.pdbqt

center_x = 5 
center_y = -1
center_z = 15  

size_x = 15
size_y = 16
size_z = 15
EOF

In [ ]:
!~/Downloads/autodock_vina_1_1_2_mac_catalina_64bit/bin/vina --config conf.txt --log log.txt

Let us next create a separate file for each of the ligands in order to visualize them in the widgets below. We will first extract the relevant lines from the `data/all.pdbqt` file into a new file that can be easily parsed. After that we will create a `model<id>.pdb` file for each model

In [ ]:
!egrep '(MODEL|ATOM|ENDMDL|VINA)' data/all.pdbqt > data/all.pdb
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.PDBParser import PDBParser

def selectModel(ofn, modelID='0'):

    class ModelSelector():
        def __init__(self, modelID=modelID):
            self.modelID = modelID

        def accept_chain(self, chain):
            return 1

        def accept_model(self, model):
            if model.get_id() == self.modelID:
                return 1
            return 0

        def accept_residue(self, residue):
            return 1

        def accept_atom(self, atom):
            return 1

    sel = ModelSelector(modelID)
    io = PDBIO()
    io.set_structure(structure)
    io.save(ofn, sel)

parser = PDBParser(QUIET=1)
structure = parser.get_structure('x', 'data/all.pdb')

for model in structure:
    print(model)
    selectModel('data/model'+str(model.id)+'.pdb',modelID=model.id)

In [ ]:
def complxvis(objeto,protein_name,ligand_name):
    mol1 = open(protein_name, 'r').read()
    mol2 = open(ligand_name, 'r').read()
    objeto.addModel(mol1,'pdb')
    objeto.setStyle({'cartoon': {'color':'spectrum'}})
    objeto.addModel(mol2,'pdb')
    objeto.setStyle({'model':1},{'stick':{}})


def complxvis2(protein_name,ligand_name):
    mview = py3Dmol.view(800, 400)  
    mol1 = open(protein_name, 'r').read()
    mol2 = open(ligand_name, 'r').read()
    mview.addModel(mol1,'pdb')
    mview.setStyle({'cartoon': {'color':'spectrum'}})
    mview.addModel(mol2,'pdb')
    mview.setStyle({'model':1},{'stick':{}})
    mview.setBackgroundColor('0xeeeeee')
    mview.zoomTo()
    mview.show()

In [ ]:
!egrep '(MODEL|VINA RESULT)' data/all.pdb
for model in structure:
    print('MODEL',model.id+1)
#complxvis2('data/R.pdb','data/model2.pdb')
    complxvis2('data/R.pdb','data/model'+str(model.id)+'.pdb')